# Kaggle Iceberg Detection Challenge

Using a ResNet with ReLU.

#### Load Dependencies

In [ ]:
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, LearningRateScheduler

from sklearn.model_selection import train_test_split

import pandas as pd

from sklearn.model_selection import train_test_split


#### Load Datasets

In [ ]:
train = pd.read_json("kaggle/datasets/iceberg/train.json")
test = pd.read_json("kaggle/datasets/iceberg/test.json")

#### Parse the Datasets

In [ ]:
n_classes = 2

X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1 + X_band_2) / 2)[:, :, :, np.newaxis]], axis = -1)

target_train = train['is_iceberg']
X_train, X_val, y_train, y_val = train_test_split(X_train, target_train, random_state = 42, train_size = 0.75)

y_train = keras.utils.to_categorical(y_train, n_classes)
y_val = keras.utils.to_categorical(y_val, n_classes)

#### Identity Block

In [ ]:
def identity_block(X, main_path_shape, filters, stage, block):
    """
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    main_path_shape -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    # Kernel Size is (1, 1), or just 1
    # Default strides is (1, 1)
    X = Conv2D(F1, kernel_size = 1, padding = 'valid', name = conv_name_base + '2a', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('ReLU_s')(X)
    
    # Second component of main path
    # Kernel Size is (main_path_shape, main_path_shape)
    # Default strides is (1, 1)
    X = Conv2D(F2, kernel_size = main_path_shape, padding = 'same', name = conv_name_base + '2b', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('ReLU_s')(X)

    # Third component of main path
    # Default strides is (1, 1)
    X = Conv2D(F3, kernel_size = 1, padding = 'valid', name = conv_name_base + '2c', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut, X])
    X = Activation('ReLU_s')(X)
    
    return X

#### Convolutional Block

In [ ]:
def convolutional_block(X, main_path_shape, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    main_path_shape -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, kernel_size = 1, strides = s, name = conv_name_base + '2a', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('ReLU_s')(X)

    # Second component of main path
    X = Conv2D(F2, kernel_size = main_path_shape, padding = 'same', name = conv_name_base + '2b', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('ReLU_s')(X)

    # Third component of main path
    X = Conv2D(F3, kernel_size = 1, padding = 'valid', name = conv_name_base + '2c', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, kernel_size = 1, strides = s, padding = 'valid', name = conv_name_base + '1', kernel_initializer = 'he_uniform')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut, X])
    X = Activation('ReLU_s')(X)
    
    return X

#### Define Custom Activation Function

In [ ]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

def relus(Z):
    e_param = 0.001
    pi = K.variable((3.14))
    m = e_param + (K.sigmoid(K.sin(Z)) - K.sigmoid(K.cos(Z)) * K.exp(K.sqrt(pi)))
    A = K.maximum(m, Z)
    return A

get_custom_objects().update({'ReLU_s': Activation(relus)})

#### Build the Model

In [ ]:
def ResNet(input_shape = (75, 75, 3)):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK -> CONVBLOCK -> IDBLOCK
    -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FC -> SOFTMAX

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(16, kernel_size = 5, strides = 2, name = 'conv1', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('ReLU_s')(X)
    X = MaxPooling2D(pool_size = 3, strides = (2, 2))(X)
    X = Dropout(0.5)(X)
    
    # Stage 2
    X = convolutional_block(X, main_path_shape = 3, filters = [32, 64, 64], stage = 2, block = 'a', s = 1)
    X = identity_block(X, 3, [32, 64, 64], stage = 2, block = 'b')
    X = Dropout(0.5)(X)
    
    # Stage 3
    X = convolutional_block(X, main_path_shape = 3, filters = [64, 64, 128], stage = 3, block = 'a', s = 2)
    X = identity_block(X, 3, [64, 64, 128], stage = 3, block = 'b')
    X = Dropout(0.5)(X)

    # Stage 4
    X = convolutional_block(X, main_path_shape = 3, filters = [128, 128, 256], stage = 4, block = 'a', s = 2)
    X = identity_block(X, 3, [128, 128, 256], stage = 4, block='b')
    X = identity_block(X, 3, [128, 128, 256], stage = 4, block='c')
    X = identity_block(X, 3, [128, 128, 256], stage = 4, block='d')
    X = identity_block(X, 3, [128, 128, 256], stage = 4, block='e')
    X = Dropout(0.5)(X)

    # Stage 5
    X = convolutional_block(X, main_path_shape = 3, filters = [265, 256, 512], stage = 5, block = 'a', s = 2)
    X = identity_block(X, 3, [265, 256, 512], stage = 5, block = 'b')
    X = identity_block(X, 3, [265, 256, 512], stage = 5, block = 'c')
    X = identity_block(X, 3, [265, 256, 512], stage = 5, block = 'd')
    X = identity_block(X, 3, [265, 256, 512], stage = 5, block = 'e')
    X = Dropout(0.5)(X)

    # AVGPOOL
    X = MaxPooling2D(pool_size = 1, name = 'max_pool')(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(n_classes, activation='softmax', name = 'fc' + str(n_classes), kernel_initializer = 'he_uniform')(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name = 'ResNet')

    return model

In [ ]:
model = ResNet()
model.summary()

#### Compile Model

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

#### Tensorboard

In [ ]:
tensorboard = TensorBoard("../logs/ResNet-iceberg-relu-20-epochs-batch-16-augmentation")

#### Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

test_datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

train_generator = train_datagen.flow(X_train, y_train, batch_size = 24)

validation_generator = test_datagen.flow(X_val, y_val, batch_size = 24)

#### Train Model

In [ ]:
# hist = model.fit_generator(train_generator,
#                            steps_per_epoch = 3000,
#                            epochs = 2,
#                            verbose = 1,
#                            validation_data = validation_generator,
#                            validation_steps = 1500)
#                            callbacks = [tensorboard])

model.fit(X_train_cv, y_train_cv,
          batch_size = 16,
          epochs = 3,
          verbose = 1,
          validation_data = (X_valid, y_valid))
#           callbacks = [tensorboard])

#### Evaluate Model

In [ ]:
score = model.evaluate(X_val, y_val, verbose = 1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#### Prepare data for Submission

In [ ]:
# X_band_test_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
# X_band_test_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
# X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis],
#                         X_band_test_2[:, :, :, np.newaxis],
#                         ((X_band_test_1 + X_band_test_2) / 2)[:, :, :, np.newaxis]], axis = -1)

# predicted_test = model.predict_proba(X_test)

#### Submit

In [ ]:
# submission = pd.DataFrame()
# submission['id'] = test['id']
# submission['is_iceberg'] = predicted_test.reshape((predicted_test.shape[0]))
# submission.to_csv('kaggle/results/iceberg/submission-cnn-iceberg-relu-20-epochs-batch-16-augmentation.csv', index = False)